<a href="https://colab.research.google.com/github/dpatel200/BIOL3340_Fall2025/blob/main/BIOL3340_finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bioinformatics Final project (Fall-2025)**

In [1]:
!apt-get -qq update
!apt-get -qq install ncbi-blast+ mafft iqtree hmmer


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Extracting templates from packages: 100%
Selecting previously unselected package fonts-lato.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../00-fonts-lato_2.0-2.1_all.deb ...
Unpacking fonts-lato (2.0-2.1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../01-netbase_6.3_all.deb ...
Unpacking netbase (6.3) ...
Selecting previously unselected package iqtree.
Preparing to unpack .../02-iqtree_2.0.7+dfsg-1_amd64.deb ...
Unpacking iqtree (2.0.7+dfsg-1) ...
Selecting previously unselected package libclone-perl.
Preparing to unpack .../03-libclone-perl_0.45-1build3_amd64.deb ...
Unpacking libclone-perl (0.45-1build3) ...
Selecting previously unselected package libdata-dump-perl.
Preparing to unpack .../04-libdata-dump-perl_1.25-1_al

In [2]:
!blastp -version
!mafft --version
!hmmscan -h | head -n 3
!iqtree2 -h | head -n 5

blastp: 2.12.0+
 Package: blast 2.12.0, build Mar  8 2022 16:19:08
v7.490 (2021/Oct/30)
# hmmscan :: search sequence(s) against a profile database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
IQ-TREE multicore version 2.0.7 for Linux 64-bit built Jan 21 2022
Developed by Bui Quang Minh, Nguyen Lam Tung, Olga Chernomor,
Heiko Schmidt, Dominik Schrempf, Michael Woodhams.

Usage: iqtree [-s ALIGNMENT] [-p PARTITION] [-m MODEL] [-t TREE] ...


# **Project Parameters**

In [3]:
# Thresholds for BLAST and alignment filtering
EVAL_THRESHOLD = 1e-10
MIN_ALIGN_LEN = 100
TOP_N = 5

print("Parameters loaded.")


Parameters loaded.


# **Download Demo Proteome (Frankia alni ACN14a)**

In [4]:
!wget -q https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/015/005/GCF_000015005.1_ASM1500v1/GCF_000015005.1_ASM1500v1_protein.faa.gz
!gunzip -f GCF_000015005.1_ASM1500v1_protein.faa.gz

#**Upload Unknown genome**

In [ ]:
from google.colab import files
import os

print("Upload your unknown genome (.faa or .fna)...")
uploaded = files.upload()

uploaded_files = list(uploaded.keys())
print("Uploaded:", uploaded_files)

UNKNOWN_FILE = uploaded_files[0]  # use first file


#**Detect File Type & Validate FASTA**

In [ ]:
def is_fasta(filename):
    with open(filename, "r") as f:
        first = f.readline().strip()
        return first.startswith(">")

if not is_fasta(UNKNOWN_FILE):
    raise ValueError("ERROR: This file does not appear to be FASTA format.")

print("FASTA format confirmed.")

# Protein or nucleotide?
ext = UNKNOWN_FILE.lower()

if ext.endswith(".faa") or ext.endswith(".fa") or ext.endswith(".fasta"):
    TYPE = "protein"
elif ext.endswith(".fna"):
    TYPE = "nucleotide"
else:
    TYPE = "unknown"

print("Detected type:", TYPE)


#**Predict Proteins if Needed (Prodigal)**

In [ ]:
if TYPE == "nucleotide":
    print("Running Prodigal for gene prediction...")
    !prodigal -i $UNKNOWN_FILE -a unknown_predicted.faa -o genes.gbk -q
    PROTEOME = "unknown_predicted.faa"
else:
    PROTEOME = UNKNOWN_FILE

print("Proteome ready:", PROTEOME)
